QUESTION 1

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [2]:
# Load the data
file_path = r'C:\Users\FahadPansota\Desktop\University\Semester 3\Financial Analytics - 632\Week 6\Data\Q1\HF_data.xlsx'
data = pd.read_excel(file_path)

In [3]:
# Clean the data by removing any rows or columns that are not needed
data_cleaned = data.drop(columns=['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 3', 'Unnamed: 4'])

Part 1 - Excess Returns

In [4]:
# Calculate excess returns for each hedge fund index by subtracting the risk-free rate from each index's return
excess_returns = data_cleaned.iloc[:, 1:].sub(data_cleaned['RF'], axis=0)

# Display the excess returns
print(excess_returns)

     Ln/Sh Eq Hedge Fund USD  Eq Mkt Ntr Hedge Fund USD  \
0                   0.009233                  -0.007967   
1                  -0.027111                   0.000006   
2                  -0.041814                  -0.005214   
3                  -0.018420                  -0.000303   
4                   0.002350                  -0.004429   
..                       ...                        ...   
217                 0.026418                   0.013335   
218                 0.004998                  -0.001553   
219                -0.004680                  -0.004139   
220                -0.045374                  -0.031906   
221                 0.003438                   0.009249   

     Ded Sh Bs Hedge Fund USD  Global Mac Hedge Fund USD  \
0                   -0.018767                  -0.001067   
1                    0.017613                  -0.058911   
2                    0.069127                  -0.045554   
3                    0.010010                  -0.0

Part 2 - Average Return, Volatility, Sharpe ratio

In [5]:
# Define equal weights for the portfolio
num_strategies = 9  # Explicitly set the number of strategies to 9
weights = np.ones(num_strategies) / num_strategies  # Create an array of equal weights (1/9)

# Compute the portfolio return for each period (month)
portfolio_returns = excess_returns.dot(weights)

# Add the date column back for reference
portfolio_returns_monthly = pd.DataFrame({'Date': data['Unnamed: 0'], 'Portfolio Return': portfolio_returns})

# Convert the 'Date' column to datetime format for easier filtering
portfolio_returns_monthly['Date'] = pd.to_datetime(portfolio_returns_monthly['Date'])

# Filter the data for the two periods and compute the cumulative returns
period1_returns = portfolio_returns_monthly[(portfolio_returns_monthly['Date'] >= '1994-01-01') & (portfolio_returns_monthly['Date'] <= '2003-12-31')]
period2_returns = portfolio_returns_monthly[(portfolio_returns_monthly['Date'] >= '2004-01-01') & (portfolio_returns_monthly['Date'] <= '2012-12-31')]

# Display the portfolio returns for each month
print(portfolio_returns_monthly)


          Date  Portfolio Return
0   1994-01-31          0.014289
1   1994-02-28         -0.011105
2   1994-03-31         -0.010778
3   1994-04-29         -0.016705
4   1994-05-31          0.003512
..         ...               ...
217 2012-02-29          0.009652
218 2012-03-30         -0.003361
219 2012-04-30         -0.003187
220 2012-05-31         -0.002115
221 2012-06-29         -0.006246

[222 rows x 2 columns]


In [6]:
# Calculate the cumulative return for each period
cumulative_return_period1 = period1_returns['Portfolio Return'].sum()
cumulative_return_period2 = period2_returns['Portfolio Return'].sum()

# Print the cumulative returns for each period
print(f"Cumulative portfolio return for 1994-2003: {cumulative_return_period1}")
print(f"Cumulative portfolio return for 2004-2012: {cumulative_return_period2}")

Cumulative portfolio return for 1994-2003: 0.4475871244707044
Cumulative portfolio return for 2004-2012: 0.19652605483436306


In [7]:
# Split the data into two sample periods
period1 = portfolio_returns_monthly.loc[portfolio_returns_monthly['Date'].between('1994-01-01', '2003-12-31')]
period2 = portfolio_returns_monthly.loc[portfolio_returns_monthly['Date'].between('2004-01-01', '2012-12-31')]

# Step 1: Compute the average of excess return for each month for both periods
avg_monthly_return_period1 = period1['Portfolio Return'].mean()
avg_monthly_return_period2 = period2['Portfolio Return'].mean()

# Step 2: Compute the average of values calculated in Step 1 for both periods
# This step is redundant as it is essentially the same as Step 1, so we'll use avg_monthly_return_period1 and avg_monthly_return_period2

# Step 3: Calculate the volatility (standard deviation) for each period
volatility_period1 = period1['Portfolio Return'].std()
volatility_period2 = period2['Portfolio Return'].std()

# Step 4: Annualize the average return
annualized_avg_return_period1 = avg_monthly_return_period1 * 12
annualized_avg_return_period2 = avg_monthly_return_period2 * 12

# Step 5: Annualize the volatility
annualized_volatility_period1 = volatility_period1 * np.sqrt(12)
annualized_volatility_period2 = volatility_period2 * np.sqrt(12)

# Compute the Sharpe ratio for each period
sharpe_ratio_period1 = (annualized_avg_return_period1) / annualized_volatility_period1
sharpe_ratio_period2 = (annualized_avg_return_period2) / annualized_volatility_period2

# Display the results
print("\n1994-2003 Period:")
print(f"Average Return: {avg_monthly_return_period1}")
print(f"Volatility: {volatility_period1}")
print(f"Annualized Average Return: {annualized_avg_return_period1}")
print(f"Annualized Volatility: {annualized_volatility_period1}")
print(f"Sharpe Ratio: {sharpe_ratio_period1}")

print("\n2004-2012 Period:")
print(f"Average Return: {avg_monthly_return_period2}")
print(f"Volatility: {volatility_period2}")
print(f"Annualized Average Return: {annualized_avg_return_period2}")
print(f"Annualized Volatility: {annualized_volatility_period2}")
print(f"Sharpe Ratio: {sharpe_ratio_period2}")


1994-2003 Period:
Average Return: 0.003729892703922537
Volatility: 0.012094636997191936
Annualized Average Return: 0.04475871244707044
Annualized Volatility: 0.041897051556477424
Sharpe Ratio: 1.0683022022858932

2004-2012 Period:
Average Return: 0.001926726027787873
Volatility: 0.014227671521679561
Annualized Average Return: 0.023120712333454477
Annualized Volatility: 0.0492860998978996
Sharpe Ratio: 0.4691122320766103


Part 3 - Optimal Weights and Sharpe Ratio

In [8]:
# Ensure column names in excess_returns are consistent
excess_returns.columns = excess_returns.columns.str.strip()

# Extract the excess returns for the period 1994-2003
period1_excess_returns = excess_returns.loc[data['Unnamed: 0'].between('1994-01-01', '2003-12-31')]

# Function to calculate the negative Sharpe ratio (return to volatility ratio)
def negative_sharpe_ratio(weights, returns):
    portfolio_return = np.dot(weights, returns.mean()) * 12
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 12, weights)))
    sharpe_ratio = portfolio_return / portfolio_volatility
    return -sharpe_ratio

# Constraints: sum of weights is 1
constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}

# Bounds for weights: between 0 and 1
bounds = [(0, 1) for _ in range(num_strategies)]

# Initial guess: equal weights
initial_guess = np.ones(num_strategies) / num_strategies

# Perform the optimization to find the optimal weights for 1994-2003
optimized_result_1994_2003 = minimize(negative_sharpe_ratio, initial_guess,
                                      args=(period1_excess_returns,),
                                      method='SLSQP', bounds=bounds, constraints=constraints)

# Get the optimal weights for 1994-2003
optimal_weights_1994_2003 = optimized_result_1994_2003.x

# Calculate the Sharpe ratio of the optimal portfolio for 1994-2003
optimal_sharpe_ratio_1994_2003 = -optimized_result_1994_2003.fun

# Display the optimal weights and Sharpe ratio for 1994-2003
print("\nOptimal Portfolio for 1994-2003 Period:")
print(f"Optimal Weights: {optimal_weights_1994_2003}")
print(f"Optimal Sharpe Ratio: {optimal_sharpe_ratio_1994_2003}")

# Compare with equally weighted portfolio
print("\nComparison with Equally Weighted Portfolio:")
print(f"Sharpe Ratio (Equally Weighted): {sharpe_ratio_period1}")
print(f"Sharpe Ratio (Optimal): {optimal_sharpe_ratio_1994_2003}")


Optimal Portfolio for 1994-2003 Period:
Optimal Weights: [4.01264122e-02 6.69843847e-01 7.19438302e-02 5.69958487e-03
 2.64820956e-15 9.43353705e-15 9.94947447e-02 1.12891581e-01
 3.51493550e-15]
Optimal Sharpe Ratio: 2.255036055162366

Comparison with Equally Weighted Portfolio:
Sharpe Ratio (Equally Weighted): 1.0683022022858932
Sharpe Ratio (Optimal): 2.255036055162366


Part 4 - Optimal Weights for next time period - Average Returns, Volatility, Sharpe Ratio

In [11]:
# Use the optimal weights found for 1994-2003
optimal_weights_2004_2012 = optimal_weights_1994_2003

# Extract the excess returns for the period 2004-2012
period2_excess_returns = excess_returns.loc[data['Unnamed: 0'].between('2004-01-01', '2012-12-31')]

# Step 1: Calculate monthly returns using optimal weights for the period 2004-2012
portfolio_returns_2004_2012_optimal = period2_excess_returns.dot(optimal_weights_2004_2012)

# Step 2: Print the monthly returns
print("\nMonthly Portfolio Returns (2004-2012) using Optimal Weights:")
print(portfolio_returns_2004_2012_optimal)



Monthly Portfolio Returns (2004-2012) using Optimal Weights:
120    0.008132
121    0.006911
122   -0.002484
123    0.000380
124   -0.000313
         ...   
217    0.010380
218   -0.000431
219   -0.005294
220   -0.018977
221    0.004485
Length: 102, dtype: float64


In [12]:
# Step 3: Calculate the average of those monthly returns
avg_monthly_return_2004_2012_optimal = portfolio_returns_2004_2012_optimal.mean()
print(f"\nAverage Monthly Return (2004-2012) using Optimal Weights: {avg_monthly_return_2004_2012_optimal}")

# Step 4: Annualize the average return by multiplying by 12
annualized_avg_return_2004_2012_optimal = avg_monthly_return_2004_2012_optimal * 12
print(f"Annualized Average Return (2004-2012) using Optimal Weights: {annualized_avg_return_2004_2012_optimal}")

# Step 5: Calculate the standard deviation of monthly returns
volatility_monthly_2004_2012_optimal = portfolio_returns_2004_2012_optimal.std()
print(f"\nMonthly Volatility (2004-2012) using Optimal Weights: {volatility_monthly_2004_2012_optimal}")

# Step 6: Annualize the standard deviation by multiplying by the square root of 12
annualized_volatility_2004_2012_optimal = volatility_monthly_2004_2012_optimal * np.sqrt(12)
print(f"Annualized Volatility (2004-2012) using Optimal Weights: {annualized_volatility_2004_2012_optimal}")

# Step 7: Compute the Sharpe ratio
sharpe_ratio_2004_2012_optimal = annualized_avg_return_2004_2012_optimal / annualized_volatility_2004_2012_optimal
print(f"\nSharpe Ratio (2004-2012) using Optimal Weights: {sharpe_ratio_2004_2012_optimal}")

# Calculate the expected return, volatility, and Sharpe ratio for an equal-weighted portfolio over the same period
equal_weights = np.ones(num_strategies) / num_strategies
portfolio_returns_2004_2012_equal = period2_excess_returns.dot(equal_weights)
avg_monthly_return_2004_2012_equal = portfolio_returns_2004_2012_equal.mean()
annualized_avg_return_2004_2012_equal = avg_monthly_return_2004_2012_equal * 12
volatility_monthly_2004_2012_equal = portfolio_returns_2004_2012_equal.std()
annualized_volatility_2004_2012_equal = volatility_monthly_2004_2012_equal * np.sqrt(12)
sharpe_ratio_2004_2012_equal = annualized_avg_return_2004_2012_equal / annualized_volatility_2004_2012_equal

# Display the results for the equal-weighted portfolio
print("\nOut-of-Sample Performance (2004-2012) using Equal Weights:")
print(f"Expected Return: {annualized_avg_return_2004_2012_equal}")
print(f"Volatility: {annualized_volatility_2004_2012_equal}")
print(f"Sharpe Ratio: {sharpe_ratio_2004_2012_equal}")


Average Monthly Return (2004-2012) using Optimal Weights: -0.0006987892817887557
Annualized Average Return (2004-2012) using Optimal Weights: -0.00838547138146507

Monthly Volatility (2004-2012) using Optimal Weights: 0.02955109570879307
Annualized Volatility (2004-2012) using Optimal Weights: 0.10236799837392044

Sharpe Ratio (2004-2012) using Optimal Weights: -0.08191496868811861

Out-of-Sample Performance (2004-2012) using Equal Weights:
Expected Return: 0.023120712333454477
Volatility: 0.0492860998978996
Sharpe Ratio: 0.4691122320766103


Optimal Weights for 2004-2012

In [14]:
# Ensure column names in excess_returns are consistent
excess_returns.columns = excess_returns.columns.str.strip()

# Extract the excess returns for the period 2004-2012
period2_excess_returns = excess_returns.loc[data['Unnamed: 0'].between('2004-01-01', '2012-12-31')]

# Function to calculate the negative Sharpe ratio (return to volatility ratio)
def negative_sharpe_ratio(weights, returns):
    portfolio_return = np.dot(weights, returns.mean()) * 12
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 12, weights)))
    sharpe_ratio = portfolio_return / portfolio_volatility
    return -sharpe_ratio

# Constraints: sum of weights cannot exceed 1
constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}

# Bounds for weights: between 0 and 1
bounds = [(0, 1) for _ in range(num_strategies)]

# Initial guess: equal weights
initial_guess = np.ones(num_strategies) / num_strategies

# Perform the optimization to find the optimal weights for 2004-2012
optimized_result_2004_2012 = minimize(negative_sharpe_ratio, initial_guess,
                                      args=(period2_excess_returns,),
                                      method='SLSQP', bounds=bounds, constraints=constraints)

# Get the optimal weights for 2004-2012
optimal_weights_2004_2012_new = optimized_result_2004_2012.x

# Calculate the Sharpe ratio of the optimal portfolio for 2004-2012
optimal_sharpe_ratio_2004_2012_new = -optimized_result_2004_2012.fun

# Display the optimal weights and Sharpe ratio for 2004-2012
print("\nOptimal Portfolio for 2004-2012 Period:")
print(f"Optimal Weights: {optimal_weights_2004_2012_new}")
print(f"Optimal Sharpe Ratio: {optimal_sharpe_ratio_2004_2012_new}")

# Compare optimal weights for 1994-2003 with 2004-2012
print("\nComparison of Optimal Weights:")
print(f"Optimal Weights (1994-2003): {optimal_weights_1994_2003}")
print(f"Optimal Weights (2004-2012): {optimal_weights_2004_2012_new}")



Optimal Portfolio for 2004-2012 Period:
Optimal Weights: [5.62981788e-16 8.57862847e-17 0.00000000e+00 9.09079282e-01
 1.72233044e-15 0.00000000e+00 9.09207185e-02 2.54668182e-15
 9.82007578e-16]
Optimal Sharpe Ratio: 1.1971930416079115

Comparison of Optimal Weights:
Optimal Weights (1994-2003): [4.01264122e-02 6.69843847e-01 7.19438302e-02 5.69958487e-03
 2.64820956e-15 9.43353705e-15 9.94947447e-02 1.12891581e-01
 3.51493550e-15]
Optimal Weights (2004-2012): [5.62981788e-16 8.57862847e-17 0.00000000e+00 9.09079282e-01
 1.72233044e-15 0.00000000e+00 9.09207185e-02 2.54668182e-15
 9.82007578e-16]


Part 5

In [15]:
# Using Equal Weights

# Define equal weights for the portfolio
equal_weights = np.ones(num_strategies) / num_strategies

# Compute the portfolio return for each period (month) from 1994-2012 using equal weights
portfolio_returns_equal_whole = excess_returns.dot(equal_weights)

# Add the date column back for reference
portfolio_returns_equal_whole_sample = pd.DataFrame({'Date': data['Unnamed: 0'], 'Portfolio Return': portfolio_returns_equal_whole})

# Convert the 'Date' column to datetime format for easier filtering
portfolio_returns_equal_whole_sample['Date'] = pd.to_datetime(portfolio_returns_equal_whole_sample['Date'])

# Display the portfolio returns for each month
print(portfolio_returns_equal_whole_sample)

          Date  Portfolio Return
0   1994-01-31          0.014289
1   1994-02-28         -0.011105
2   1994-03-31         -0.010778
3   1994-04-29         -0.016705
4   1994-05-31          0.003512
..         ...               ...
217 2012-02-29          0.009652
218 2012-03-30         -0.003361
219 2012-04-30         -0.003187
220 2012-05-31         -0.002115
221 2012-06-29         -0.006246

[222 rows x 2 columns]


In [16]:
# Calculate the average of those monthly returns
avg_monthly_return_equal_whole = portfolio_returns_equal_whole_sample['Portfolio Return'].mean()
print(f"\nAverage Monthly Return (1994-2012) using Equal Weights: {avg_monthly_return_equal_whole}")

# Annualize the average return by multiplying by 12
annualized_avg_return_equal_whole = avg_monthly_return_equal_whole * 12
print(f"Annualized Average Return (1994-2012) using Equal Weights: {annualized_avg_return_equal_whole}")

# Calculate the standard deviation of monthly returns
volatility_monthly_equal_whole = portfolio_returns_equal_whole_sample['Portfolio Return'].std()
print(f"\nMonthly Volatility (1994-2012) using Equal Weights: {volatility_monthly_equal_whole}")

# Annualize the standard deviation by multiplying by the square root of 12
annualized_volatility_equal_whole = volatility_monthly_equal_whole * np.sqrt(12)
print(f"Annualized Volatility (1994-2012) using Equal Weights: {annualized_volatility_equal_whole}")

# Compute the Sharpe ratio
sharpe_ratio_equal_whole = annualized_avg_return_equal_whole / annualized_volatility_equal_whole
print(f"\nSharpe Ratio (1994-2012) using Equal Weights: {sharpe_ratio_equal_whole}")


Average Monthly Return (1994-2012) using Equal Weights: 0.0029014107175903943
Annualized Average Return (1994-2012) using Equal Weights: 0.034816928611084735

Monthly Volatility (1994-2012) using Equal Weights: 0.013118276137666791
Annualized Volatility (1994-2012) using Equal Weights: 0.045443041556314595

Sharpe Ratio (1994-2012) using Equal Weights: 0.7661663352339299


In [17]:
# Using Optimal Weights from 1994-2003

# Use the optimal weights found for 1994-2003
optimal_weights_1994_2003 = optimal_weights_1994_2003

# Compute the portfolio return for each period (month) from 1994-2012 using optimal weights from 1994-2003
portfolio_returns_optimal_whole = excess_returns.dot(optimal_weights_1994_2003)

# Add the date column back for reference
portfolio_returns_optimal_whole_sample = pd.DataFrame({'Date': data['Unnamed: 0'], 'Portfolio Return': portfolio_returns_optimal_whole})

# Convert the 'Date' column to datetime format for easier filtering
portfolio_returns_optimal_whole_sample['Date'] = pd.to_datetime(portfolio_returns_optimal_whole_sample['Date'])

# Display the portfolio returns for each month
print(portfolio_returns_optimal_whole_sample)

          Date  Portfolio Return
0   1994-01-31         -0.002842
1   1994-02-28         -0.000648
2   1994-03-31         -0.003431
3   1994-04-29         -0.004425
4   1994-05-31         -0.003302
..         ...               ...
217 2012-02-29          0.010380
218 2012-03-30         -0.000431
219 2012-04-30         -0.005294
220 2012-05-31         -0.018977
221 2012-06-29          0.004485

[222 rows x 2 columns]


In [18]:
# Using Optimal Weights from 1994-2003

# Use the optimal weights found for 1994-2003
optimal_weights_1994_2003 = optimal_weights_1994_2003

# Compute the portfolio return for each period (month) from 1994-2012 using optimal weights from 1994-2003
portfolio_returns_optimal_whole = excess_returns.dot(optimal_weights_1994_2003)

# Add the date column back for reference
portfolio_returns_optimal_whole_sample = pd.DataFrame({'Date': data['Unnamed: 0'], 'Portfolio Return': portfolio_returns_optimal_whole})

# Convert the 'Date' column to datetime format for easier filtering
portfolio_returns_optimal_whole_sample['Date'] = pd.to_datetime(portfolio_returns_optimal_whole_sample['Date'])

# Calculate the average of those monthly returns
avg_monthly_return_optimal_whole = portfolio_returns_optimal_whole_sample['Portfolio Return'].mean()
print(f"\nAverage Monthly Return (1994-2012) using Optimal Weights from 1994-2003: {avg_monthly_return_optimal_whole}")

# Annualize the average return by multiplying by 12
annualized_avg_return_optimal_whole = avg_monthly_return_optimal_whole * 12
print(f"Annualized Average Return (1994-2012) using Optimal Weights from 1994-2003: {annualized_avg_return_optimal_whole}")

# Calculate the standard deviation of monthly returns
volatility_monthly_optimal_whole = portfolio_returns_optimal_whole_sample['Portfolio Return'].std()
print(f"\nMonthly Volatility (1994-2012) using Optimal Weights from 1994-2003: {volatility_monthly_optimal_whole}")

# Annualize the standard deviation by multiplying by the square root of 12
annualized_volatility_optimal_whole = volatility_monthly_optimal_whole * np.sqrt(12)
print(f"Annualized Volatility (1994-2012) using Optimal Weights from 1994-2003: {annualized_volatility_optimal_whole}")

# Compute the Sharpe ratio
sharpe_ratio_optimal_whole = annualized_avg_return_optimal_whole / annualized_volatility_optimal_whole
print(f"\nSharpe Ratio (1994-2012) using Optimal Weights from 1994-2003: {sharpe_ratio_optimal_whole}")



Average Monthly Return (1994-2012) using Optimal Weights from 1994-2003: 0.0020392670653705163
Annualized Average Return (1994-2012) using Optimal Weights from 1994-2003: 0.024471204784446195

Monthly Volatility (1994-2012) using Optimal Weights from 1994-2003: 0.020729792600532957
Annualized Volatility (1994-2012) using Optimal Weights from 1994-2003: 0.07181010802897689

Sharpe Ratio (1994-2012) using Optimal Weights from 1994-2003: 0.34077660452163017


In [19]:
# Comparison of Results

print("\nComparison of Whole Sample (1994-2012) Performance:")
print(f"\nEqual Weights Portfolio (1994-2012):")
print(f"Expected Return: {annualized_avg_return_equal_whole}")
print(f"Volatility: {annualized_volatility_equal_whole}")
print(f"Sharpe Ratio: {sharpe_ratio_equal_whole}")

print(f"\nOptimal Weights Portfolio (1994-2012):")
print(f"Expected Return: {annualized_avg_return_optimal_whole}")
print(f"Volatility: {annualized_volatility_optimal_whole}")
print(f"Sharpe Ratio: {sharpe_ratio_optimal_whole}")



Comparison of Whole Sample (1994-2012) Performance:

Equal Weights Portfolio (1994-2012):
Expected Return: 0.034816928611084735
Volatility: 0.045443041556314595
Sharpe Ratio: 0.7661663352339299

Optimal Weights Portfolio (1994-2012):
Expected Return: 0.024471204784446195
Volatility: 0.07181010802897689
Sharpe Ratio: 0.34077660452163017


Part 6

Yes. Given the higher expected return, lower volatility, and superior Sharpe Ratio, the equal-weighted portfolio appears to be the better choice for allocating capital across strategies for the period 1994-2012. It offers a more balanced risk-return trade-off compared to the portfolio using optimal weights from 1994-2003.

END OF QUESTION